In [1]:
#Import libraries
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from sklearn.model_selection import train_test_split
from scipy import stats

%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [3]:
import pandas as pd

#Read input data
df_train = pd.read_csv('/content/drive/MyDrive/ML/housing-prices-ml-project/data/raw/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/ML/housing-prices-ml-project/data/raw/test.csv')

#df_train['Data_Split'] = 'train'
#df_test['Data_Split'] = 'test'
#df = pd.concat([df_train, df_test])

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ML/housing-prices-ml-project/data/raw/train.csv'

In [ ]:
df_train.info()

In [ ]:
#Find any missing values in the dataset
df_train.isnull().sum()

In [ ]:
###Removing Irrelevant coloumn
columns_to_keep = ['SalePrice','MoSold','YrSold', 'MSZoning','LotArea','Alley','Utilities','BldgType','OverallCond','YearBuilt','MasVnrArea','TotalBsmtSF', 'Heating', 'CentralAir', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'Fireplaces', 'GarageYrBlt', 'PoolArea', 'WoodDeckSF', 'OpenPorchSF', ]
df_train1 = df_train[columns_to_keep]

In [ ]:
df_train1.isnull().sum()

In [ ]:
df_train1.GarageYrBlt.fillna(0)
df_train1.GarageYrBlt.dropna()

In [ ]:
df_train1.Alley.fillna(0)
df_train1.Alley.dropna()

In [ ]:
# label several label with specific grading system.
def SC_LabelEncoder3(text):
    if text == "Grvl":
        return 1
    elif text == "Pave":
        return 2
    else:
        return 0

def SC_LabelEncoder2(text):
    if text == "A":
        return 1
    elif text == "C":
        return 2
    elif text == "FV":
        return 3
    elif text == "RP":
        return 4
    elif text == "I":
        return 5
    elif text == "RH":
        return 6
    elif text == "RL":
        return 6
    elif text == "RM":
        return 7
    else:
        return 0

def SC_LabelEncoder(text):
    if text == "AllPub":
        return 4
    elif text == "NoSewr":
        return 3
    elif text == "NoSeWa":
        return 2
    elif text == "ELO":
        return 1
    else:
        return  0

def SC_LabelEncoder4(text):
    if text == "1Fam":
        return 1
    elif text == "2FmCon":
        return 2
    elif text == "Duplx":
        return 3
    elif text == "TwnhsE":
        return 4
    elif text == "TwnhsI":
        return 4
    else:
        return 0

def SC_LabelEncoder5(text):

    if text == "Floor":
        return 1
    elif text == "GasA":
        return 2
    elif text == "GasW":
        return 3
    elif text == "Grav":
        return 4
    elif text == "OthW":
        return 5
    elif text == "Wall":
        return 6
    else:
        return 0

def SC_LabelEncoder6(text):
    if text == "Y":
        return 1
    else:
        return 0

df_train1["Alley"] = df_train1["Alley"].apply(SC_LabelEncoder3)
df_train1["MSZoning"] = df_train1["MSZoning"].apply(SC_LabelEncoder2)
df_train1["Utilities"] = df_train1["Utilities"].apply(SC_LabelEncoder)
df_train1["BldgType"] = df_train1["BldgType"].apply(SC_LabelEncoder4)
df_train1["Heating"] = df_train1["Heating"].apply(SC_LabelEncoder5)
df_train1["CentralAir"] = df_train1["CentralAir"].apply(SC_LabelEncoder6)

In [ ]:
df_train1.head(10)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, Latex

sns.set_style('whitegrid')

from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import f1_score

In [ ]:
X, y = df_train1.drop("SalePrice", axis=1), df_train1["SalePrice"]
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.20, random_state=0)

### Random forest

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, random_state=42) # n_estimators is the number of trees
rfc.fit(X_train, y_train)

In [ ]:
# Make predictions
y_pred_clf = rfc.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred_clf)
print(f"Classification Accuracy: {accuracy}")

In [ ]:
from pathlib import Path
def apply_domain_cleaning(df):
    df = df.copy()
    # 1) NA -> "None"
    none_cols = ["Alley","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2",
                 "FireplaceQu","GarageType","GarageFinish","GarageQual","GarageCond",
                 "PoolQC","Fence","MiscFeature","MasVnrType"]
    for c in none_cols:
        if c in df.columns:
            df[c] = df[c].fillna("None")

    # 2) Zero fill
    zero_cols = ["MasVnrArea","GarageCars","GarageArea","TotalBsmtSF",
                 "BsmtFinSF1","BsmtFinSF2","BsmtUnfSF","BsmtFullBath","BsmtHalfBath","PoolArea"]
    for c in zero_cols:
        if c in df.columns:
            df[c] = df[c].fillna(0)

    # 3) LotFrontage median by neighborhood
    if "LotFrontage" in df.columns and "Neighborhood" in df.columns:
        df["LotFrontage"] = df.groupby("Neighborhood")["LotFrontage"].transform(
            lambda s: s.fillna(s.median())
        )

    # 4) GarageYrBlt
    if "GarageYrBlt" in df.columns:
        if "GarageType" in df.columns:
            df.loc[df["GarageType"]=="None", "GarageYrBlt"] = df.loc[df["GarageType"]=="None", "GarageYrBlt"].fillna(0)
        if "YearBuilt" in df.columns:
            df["GarageYrBlt"] = df["GarageYrBlt"].fillna(df["YearBuilt"])

    # 5) MSZoning in test
    if "MSZoning" in df.columns and "Neighborhood" in df.columns:
        df["MSZoning"] = df.groupby("Neighborhood")["MSZoning"].transform(
            lambda s: s.fillna(s.mode().iloc[0] if not s.mode().empty else "RL")
        )

    # 6) Electrical mode
    if "Electrical" in df.columns:
        df["Electrical"] = df["Electrical"].fillna(df["Electrical"].mode()[0])

    # 7) Functional NA->Typ
    if "Functional" in df.columns:
        df["Functional"] = df["Functional"].fillna("Typ")

    # 8) Drop Utilities
    if "Utilities" in df.columns:
        df = df.drop(columns=["Utilities"])

    # 9) Feature engineering
    if set(["GrLivArea","TotalBsmtSF"]).issubset(df.columns):
        df["TotalSF"] = df["GrLivArea"] + df["TotalBsmtSF"]
    if set(["FullBath","HalfBath","BsmtFullBath","BsmtHalfBath"]).issubset(df.columns):
        df["Baths_Total"] = df["FullBath"] + 0.5*df["HalfBath"] + df["BsmtFullBath"] + 0.5*df["BsmtHalfBath"]
    if set(["YrSold","YearBuilt"]).issubset(df.columns):
        df["Age_Sold"] = df["YrSold"] - df["YearBuilt"]
    if set(["YrSold","YearRemodAdd"]).issubset(df.columns):
        df["Age_Renov"] = df["YrSold"] - df["YearRemodAdd"]
    if set(["WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch"]).issubset(df.columns):
        df["PorchSF_Total"] = df[["WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch"]].sum(axis=1)
    return df

def drop_grliv_outliers(df):
    return df[~((df["GrLivArea"]>4000) & (df["SalePrice"]<300000))].copy()

X_train = apply_domain_cleaning(df_train)
X_test  = apply_domain_cleaning(df_test)

print(X_train.shape, X_test.shape)
X_train.head()

(1460, 85) (1459, 84)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,...,MoSold,YrSold,SaleType,SaleCondition,SalePrice,TotalSF,Baths_Total,Age_Sold,Age_Renov,PorchSF_Total
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,Inside,...,2,2008,WD,Normal,208500,2566,3.5,5,5,61
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,FR2,...,5,2007,WD,Normal,181500,2524,2.5,31,31,298
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,Inside,...,9,2008,WD,Normal,223500,2706,3.5,7,6,42
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,Corner,...,2,2006,WD,Abnorml,140000,2473,2.0,91,36,307
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,FR2,...,12,2008,WD,Normal,250000,3343,3.5,8,8,276


In [ ]:
# List of non-numeric and categorical variables to drop
variables_to_drop = [
    'MSZoning',    # Categorical variable that needs encoding
    'Street',      # Categorical variable that needs encoding
    'Alley',       # Categorical variable that needs encoding
    'LotShape',    # Categorical variable that needs encoding
    'LandContour', # Categorical variable that needs encoding
    'LotConfig',   # Categorical variable that needs encoding
    'LandSlope',   # Categorical variable that needs encoding
    'Neighborhood', # Categorical variable that needs encoding
    'Condition1',  # Categorical variable that needs encoding
    'Condition2',  # Categorical variable that needs encoding
    'BldgType',    # Categorical variable that needs encoding
    'HouseStyle',  # Categorical variable that needs encoding
    'RoofStyle',   # Categorical variable that needs encoding
    'RoofMatl',    # Categorical variable that needs encoding
    'Exterior1st', # Categorical variable that needs encoding
    'Exterior2nd', # Categorical variable that needs encoding
    'MasVnrType',  # Categorical variable that needs encoding
    'ExterQual',   # Categorical variable that needs encoding
    'ExterCond',   # Categorical variable that needs encoding
    'Foundation',  # Categorical variable that needs encoding
    'BsmtQual',    # Categorical variable that needs encoding
    'BsmtCond',    # Categorical variable that needs encoding
    'BsmtExposure', # Categorical variable that needs encoding
    'BsmtFinType1', # Categorical variable that needs encoding
    'BsmtFinType2', # Categorical variable that needs encoding
    'Heating',     # Categorical variable that needs encoding
    'HeatingQC',   # Categorical variable that needs encoding
    'CentralAir',  # Categorical variable that needs encoding
    'Electrical',  # Categorical variable that needs encoding
    'KitchenQual', # Categorical variable that needs encoding
    'Functional',  # Categorical variable that needs encoding
    'FireplaceQu', # Categorical variable that needs encoding
    'GarageType',  # Categorical variable that needs encoding
    'GarageFinish', # Categorical variable that needs encoding
    'GarageQual',  # Categorical variable that needs encoding
    'GarageCond',  # Categorical variable that needs encoding
    'PavedDrive',  # Categorical variable that needs encoding
    'PoolQC',      # Categorical variable that needs encoding
    'Fence',       # Categorical variable that needs encoding
    'MiscFeature', # Categorical variable that needs encoding
    'SaleType',    # Categorical variable that needs encoding
    'SaleCondition' # Categorical variable that needs encoding
]

# Drop the identified variables
X_train = X_train.drop(columns=variables_to_drop)
X_test = X_test.drop(columns=variables_to_drop)

In [ ]:
X_train

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,PoolArea,MiscVal,MoSold,YrSold,SalePrice,TotalSF,Baths_Total,Age_Sold,Age_Renov,PorchSF_Total
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,0,2,2008,208500,2566,3.5,5,5,61
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,0,0,5,2007,181500,2524,2.5,31,31,298
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,0,9,2008,223500,2706,3.5,7,6,42
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,0,2,2006,140000,2473,2.0,91,36,307
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,0,0,12,2008,250000,3343,3.5,8,8,276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,62.0,7917,6,5,1999,2000,0.0,0,...,0,0,8,2007,175000,2600,2.5,8,7,40
1456,1457,20,85.0,13175,6,6,1978,1988,119.0,790,...,0,0,2,2010,210000,3615,3.0,32,22,349
1457,1458,70,66.0,9042,7,9,1941,2006,0.0,275,...,0,2500,5,2010,266500,3492,2.0,69,4,60
1458,1459,20,68.0,9717,5,6,1950,1996,0.0,49,...,0,0,4,2010,142125,2156,2.0,60,14,478


In [ ]:
print(list(X_train.columns))

['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice', 'TotalSF', 'Baths_Total', 'Age_Sold', 'Age_Renov', 'PorchSF_Total']


In [ ]:
X_train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,PoolArea,MiscVal,MoSold,YrSold,SalePrice,TotalSF,Baths_Total,Age_Sold,Age_Renov,PorchSF_Total
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.199658,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.117123,443.639726,...,2.758904,43.489041,6.321918,2007.815753,180921.195890,2572.893151,2.210616,36.547945,22.950000,181.329452
std,421.610009,42.300571,22.431902,9981.264932,1.382997,1.112799,30.202904,20.645407,180.731373,456.098091,...,40.177307,496.123024,2.703626,1.328095,79442.502883,823.598492,0.785399,30.250152,20.640653,156.656097
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,2006.000000,34900.000000,334.000000,1.000000,0.000000,-1.000000,0.000000
25%,365.750000,20.000000,60.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,5.000000,2007.000000,129975.000000,2014.000000,2.000000,8.000000,4.000000,45.000000
50%,730.500000,50.000000,70.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,0.000000,6.000000,2008.000000,163000.000000,2479.000000,2.000000,35.000000,14.000000,164.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,164.250000,712.250000,...,0.000000,0.000000,8.000000,2009.000000,214000.000000,3008.500000,2.500000,54.000000,41.000000,266.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,738.000000,15500.000000,12.000000,2010.000000,755000.000000,11752.000000,6.000000,136.000000,60.000000,1027.000000


In [ ]:
# Save the DataFrame as a CSV file
X_train.to_csv('/content/drive/MyDrive/ML/housing-prices-ml-project/data/processed/train.csv', index=False)

# Save the DataFrame as a CSV file
X_test.to_csv('/content/drive/MyDrive/ML/housing-prices-ml-project/data/processed/test.csv', index=False)